# Рекомендательные системы

**Описание задачи**

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит.

Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать **recall@k** и **precision@k**.

По имеющимся данным можно построить простые бейзлайны: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок.

Эти бейзлайны, с одной стороны, могут помочь грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

**Входные данные**

Имеется две выборки с пользовательскими сессиями - **id**-шниками просмотренных и **id**-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: **id** просмотренных товаров через *`,`* затем идёт *`;`* после чего следуют **id** купленных товаров (если такие имеются), разделённые запятой. Например, **1,2,3,4;** или **1,2,3,4;5,6**.

Гарантируется, что среди **id** купленных товаров все различные.

**Особенности**

- Сессии, в которых пользователь ничего не купил, исключаются из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Число рекомендуемых товаров всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и **k** в **recall@k** / **precision@k**.

**Необходимо**

1. Реализовать два алгоритма рекомендаций:
    - сортировка просмотренных **id** по популярности (частота появления в просмотренных),
    - сортировка просмотренных **id** по покупаемости (частота появления в покупках).
2. Для данных алгоритмов оценить среднюю полноту и среднюю точность при рекомендации 1 и 5 товаров **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5**.

**1. Загрузка данных.**

In [1]:
foldef_path = r'C:\Users\Xiaomi\Desktop\Data Science\5. Прикладные задачи анализа данных'
train_file_name = 'coursera_sessions_train.txt'
test_file_name = 'coursera_sessions_test.txt'

train_file = foldef_path + '\\' + train_file_name
test_file = foldef_path + '\\' + test_file_name

In [2]:
with open(train_file, 'r', encoding='utf-8') as file:
    train_data = file.read()

train_data = list(map(lambda x: x.strip(), train_data.strip().split('\n')))

In [3]:
with open(test_file, 'r', encoding='utf-8') as file:
    test_data = file.read()

test_data = list(map(lambda x: x.strip(), test_data.strip().split('\n')))

In [4]:
print('train_data:', len(train_data))
print('test_data:', len(test_data))

train_data: 50000
test_data: 50000


**2. Определение частоты появления `id` в просмотренных и в купленных.**

In [5]:
# Число появления id в просмотренных и в купленных

viewed_id = dict()
purchased_id = dict()

for session_data in train_data:
    
    viewed_id_list = session_data.split(';')[0].strip()
    purchased_id_list = session_data.split(';')[1].strip()
    
    for items_id in viewed_id_list.split(','):
        viewed_id.setdefault(items_id.strip(), 1)
        viewed_id[items_id.strip()]+=1
    
    for items_id in purchased_id_list.split(','):
        purchased_id.setdefault(items_id.strip(), 1)
        purchased_id[items_id.strip()]+=1

del purchased_id['']

In [6]:
# Частоты появления id в просмотренных и в купленных

viewed_id_frequency = dict()
purchased_id_frequency = dict()

viewed_id_sum = sum(viewed_id.values())
purchased_id_sum = sum(purchased_id.values())

for item, count in zip(viewed_id.keys(), viewed_id.values()):
    viewed_id_frequency[item] = count / viewed_id_sum

for item, count in zip(purchased_id.keys(), purchased_id.values()):
    purchased_id_frequency[item] = count / purchased_id_sum

In [74]:
# Сортировка просмотренных id по популярности (частота появления в просмотренных)
# viewed_id_frequency_sorted_dict = sorted(viewed_id_frequency.items(), key=lambda x: x[1], reverse=True)
# viewed_id_frequency_sorted_list = sorted(viewed_id_frequency, key=viewed_id_frequency.get, reverse=True)

# Сортировка просмотренных id по покупаемости (частота появления в покупках)
# purchased_id_frequency_sorted_dict = sorted(purchased_id_frequency.items(), key=lambda x: x[1], reverse=True)
# purchased_id_frequency_sorted_list = sorted(purchased_id_frequency, key=purchased_id_frequency.get, reverse=True)

**3. Подготовка рекомендаций и их оценка.**

In [7]:
# AveragePrecision@k - Доля купленных товаров из числа рекомендованных при показе k рекомендаций
# AverageRecall@k - Доля товаров купленных по рекомендации при показе k рекомендаций

def get_Average_Precision_and_Recall(k, data, recommended_dict):
    """Принимает датасет и число рекомендаций k, возвращает AveragePrecision@k и AverageRecall@k"""

    Precision_list = list()
    Recall_list = list()

    for session_data in data:
        
        viewed_id_list = session_data.split(';')[0].strip()
        purchased_id_list = session_data.split(';')[1].strip()

        if not purchased_id_list:
            continue
        
        viewed_id_list = list(map(lambda x: x.strip(), viewed_id_list.split(',')))
        purchased_id_list = list(map(lambda x: x.strip(), purchased_id_list.split(',')))
        
        # Словарь с персональными рекомендациями (id : вероятность покупки)         
        personal_recommended_dict = dict()
        
        for viewed_id in viewed_id_list:
            
            try:
                personal_recommended_dict[viewed_id] = recommended_dict[viewed_id]
                
            except Exception:
                personal_recommended_dict[viewed_id] = 0
        
        # Отсортированный список персональных рекомендаций        
        personal_recommended_sorted_list = sorted(personal_recommended_dict, 
                                                  key=personal_recommended_dict.get, 
                                                  reverse=True)
        
        
        count_good_recommendation = 0
        
        for recommended_id in personal_recommended_sorted_list[:k]:
            if recommended_id in purchased_id_list:
                count_good_recommendation += 1

        Precision = count_good_recommendation / k
        Precision_list.append(Precision)

        Recall = count_good_recommendation / len(purchased_id_list)
        Recall_list.append(Recall)
        
    AveragePrecision = sum(Precision_list) / len(Precision_list)
    AverageRecall = sum(Recall_list) / len(Recall_list)
    
    return AveragePrecision, AverageRecall

In [9]:
AveragePrecision_1, AverageRecall_1 = get_Average_Precision_and_Recall(1, train_data, viewed_id_frequency)
AveragePrecision_5, AverageRecall_5 = get_Average_Precision_and_Recall(5, train_data, viewed_id_frequency)

print('Рекомендации по частоте просмотров товаров - качество на обучающей выборке\n')

print('AverageRecall@1 =', AverageRecall_1)
print('AveragePrecision@1 =', AveragePrecision_1)
print('AverageRecall@5 =', AverageRecall_5)
print('AveragePrecision@5 =', AveragePrecision_5)

Рекомендации по частоте просмотров товаров - качество на обучающей выборке

AverageRecall@1 = 0.4426343165949593
AveragePrecision@1 = 0.5121951219512195
AverageRecall@5 = 0.8246918247126122
AveragePrecision@5 = 0.21252771618625918


In [10]:
AveragePrecision_1, AverageRecall_1 = get_Average_Precision_and_Recall(1, test_data, viewed_id_frequency)
AveragePrecision_5, AverageRecall_5 = get_Average_Precision_and_Recall(5, test_data, viewed_id_frequency)

print('Рекомендации по частоте просмотров товаров - качество на тестовой выборке\n')

print('AverageRecall@1 =', AverageRecall_1)
print('AveragePrecision@1 =', AveragePrecision_1)
print('AverageRecall@5 =', AverageRecall_5)
print('AveragePrecision@5 =', AveragePrecision_5)

Рекомендации по частоте просмотров товаров - качество на тестовой выборке

AverageRecall@1 = 0.41733266203252534
AveragePrecision@1 = 0.48130968622100956
AverageRecall@5 = 0.8000340663538579
AveragePrecision@5 = 0.2037653478854079


In [11]:
AveragePrecision_1, AverageRecall_1 = get_Average_Precision_and_Recall(1, train_data, purchased_id_frequency)
AveragePrecision_5, AverageRecall_5 = get_Average_Precision_and_Recall(5, train_data, purchased_id_frequency)

print('Рекомендации по частоте покупок товаров - качество на обучающей выборке\n')

print('AverageRecall@1 =', AverageRecall_1)
print('AveragePrecision@1 =', AveragePrecision_1)
print('AverageRecall@5 =', AverageRecall_5)
print('AveragePrecision@5 =', AveragePrecision_5)

Рекомендации по частоте покупок товаров - качество на обучающей выборке

AverageRecall@1 = 0.6884494924267653
AveragePrecision@1 = 0.8037694013303769
AverageRecall@5 = 0.9263073024228787
AveragePrecision@5 = 0.2525498891352649


In [12]:
AveragePrecision_1, AverageRecall_1 = get_Average_Precision_and_Recall(1, test_data, purchased_id_frequency)
AveragePrecision_5, AverageRecall_5 = get_Average_Precision_and_Recall(5, test_data, purchased_id_frequency)

print('Рекомендации по частоте покупок товаров - качество на тестовой выборке\n')

print('AverageRecall@1 =', AverageRecall_1)
print('AveragePrecision@1 =', AveragePrecision_1)
print('AverageRecall@5 =', AverageRecall_5)
print('AveragePrecision@5 =', AveragePrecision_5))

Рекомендации по частоте покупок товаров - качество на тестовой выборке

AverageRecall@1 = 0.4606201666660294
AveragePrecision@1 = 0.5276944065484311
AverageRecall@5 = 0.8201874337490194
AveragePrecision@5 = 0.21009549795362173


Из полученных данных видно:

При построении рекомендаций по частоте просмотров товаров, качество на тесте: 
    - AverageRecall@1 = 0.417
    - AveragePrecision@1 = 0.481
    - AverageRecall@5 = 0.800
    - AveragePrecision@5 = 0.204

Полнота (доля купленных товаров из числа рекомендованных при показе k рекомендаций) растет с увеличением числа рекомендуемых товаров с 1 до 5, точность (доля товаров купленных по рекомендации при показе k рекомендаций) падает. 

При построении рекомендаций по частоте покупок товаров, качество на тесте: 
    - AverageRecall@1 = 0.461
    - AveragePrecision@1 = 0.528
    - AverageRecall@5 = 0.821
    - AveragePrecision@5 = 0.210

Полнота растет с увеличением числа рекомендуемых товаров с 1 до 5, точность падает.

При этом по всем параметрам построение рекомендаций по частоте покупок товаров является выигрышным.